In [1]:
import os

# Retrieve the environment variable
secret_value = os.getenv('MY_SECRET')

# Print the value
if secret_value:
    print(f"MY_SECRET: {secret_value}")
else:
    print("MY_SECRET is not set.")


MY_SECRET is not set.


In [2]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [3]:
# Loading targets into stk from file.
stk_object.set_sim_time(days = 0,hours=1)
stk_object.Target_Loader("../../Input_Files/Target_Packages/Targets_15.txt")

In [ ]:
class Optimizer2:
    def __init__(self, stk_object, n_pop, n_gen, weights=(7.0,-5.0,-5.0)):
        self.stk_object = stk_object
        self.n_pop = n_pop
        self.n_gen = n_gen
        creator.create("FitnessMax", base.Fitness, weights=weights)
        creator.create("Satellite", list, fitness=creator.FitnessMax)
        self.lower = [575, 87, 30, 0, 3, 1]
        self.upper = [630, 95, 150, 30, 12, 12]

        # Registering variables to the satellite
        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_alt", random.randint, self.lower[0], self.upper[0])
        self.toolbox.register("attr_inc", random.randint, self.lower[1], self.upper[1])
        self.toolbox.register("attr_initial_raan", random.randint, self.lower[2], self.upper[2])
        self.toolbox.register("attr_delta_raan", random.randint, self.lower[3], self.upper[3])
        self.toolbox.register("att_num_sats", random.randint, self.lower[4], self.upper[4])

        # Registering satellite to the model without registering attr_num_planes separately
        def satellite_init():
            num_sats = self.toolbox.att_num_sats()
            # Generate number of planes that is <= num_sats
            num_planes = random.randint(1, num_sats) 
            return creator.Satellite([  
                self.toolbox.attr_alt(),
                self.toolbox.attr_inc(),
                self.toolbox.attr_initial_raan(),
                self.toolbox.attr_delta_raan(),
                num_sats,
                num_planes
            ])

        self.toolbox.register("satellite", satellite_init)

        # Registering tools for the algorithm
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.satellite)
        self.toolbox.register("evaluate", self.cost_function)
        self.toolbox.register("mate", tools.cxUniform, indpb=0.5)
        self.toolbox.register("mutate", tools.mutUniformInt, low=self.lower, up=self.upper, indpb=0.5)
        self.toolbox.register("select", tools.selTournament, tournsize=int(self.n_pop // 2))
        self.stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean, axis=0)
        self.stats.register("std", np.std, axis=0)
        self.stats.register("min", np.min, axis=0)
        self.stats.register("max", np.max, axis=0)


        def repair_individual(individual):
            if individual[5] > individual[4]:  # if Num_Planes > Num_Sats
                individual[5] = random.randint(1, individual[4])  # Set Num_Planes = Num_Sats

        self.toolbox.register("repair", repair_individual)


    def run(self,read=False,enable_print=False):
        self.enable_print = enable_print
        self.fits = []
        CXPB = 0.6;MUTPB=0.4
        self.g = 0
        # Creating a population to evolve
        pop = self.toolbox.population(n=self.n_pop)
        i = np.random.randint(0,self.n_pop)
        if read:
            for idx in range(4):
                pop[i][idx] = get_ind(4)[idx]

        fitnesses = list(map(self.toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        hof = tools.HallOfFame(5)
        hof.update(pop)

        self.fits.append([ind.fitness.values for ind in pop])
        record = self.stats.compile(pop)

        clear_output(wait=False)
        print("-- Generation %i --" % self.g)
        print(pd.DataFrame(record))

        with open(f"../../Pop_Over_Gen/pop_gen.csv","w") as file:
            file.write("Gen,Pop,Alt,Inc,Initial_Raan,Delta_Raan,Num_Sats,Num_Planes,Avg_Percentage,\n")
            for i in range(self.n_pop):
                file.write(f"{self.g},{i},")
                for idx in range(6):
                    file.write(f"{pop[i][idx]},")
                for fit in pop[i].fitness.getValues():
                    file.write(f"{fit},")
                file.write("\n")
        # Begin the evolution
        while self.g < self.n_gen:
            # clear_output(wait=True)
            self.g = self.g + 1

            # A new generation
            # Select the next generation individuals
            offspring = self.toolbox.select(pop, len(pop))
            # Clone the selected individuals
            offspring = list(map(self.toolbox.clone, offspring))
            # Apply crossover and mutation on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < CXPB:
                    self.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values

            for mutant in offspring:
                if random.random() < MUTPB:
                    self.toolbox.mutate(mutant)
                    del mutant.fitness.values

            for individual in offspring:
                self.toolbox.repair(individual)

            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(self.toolbox.evaluate, invalid_ind)

            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
            pop[:] = offspring
            hof.update(pop)

            self.fits.append([ind.fitness.values for ind in pop])
            record = self.stats.compile(pop)

            clear_output(wait=False)
            print("-- Generation %i --" % self.g)
            print(pd.DataFrame(record))
            
            with open(f"../../Pop_Over_Gen/pop_gen.csv","a") as file:
                for i in range(self.n_pop):
                    file.write(f"{self.g},{i},")
                    for idx in range(6):
                        file.write(f"{pop[i][idx]},")
                    for fit in pop[i].fitness.getValues():
                        file.write(f"{fit},")
                    file.write("\n")
        return hof
    
    def cost_function(self,Individual=[0,0,0,0,0,0],write=True):
        n_planes = Individual[5]
        n_sats = Individual[4]

        gen = self.g

        if n_planes > n_sats:
            n_planes = n_sats
        if write:
            self.Load_Individual(Individual)
        self.stk_object.Satellite_Loader(f'../../Input_Files/Satellites_File.txt')
        self.stk_object.Compute_AzEl(self.enable_print)
        percentages = np.array([100*np.count_nonzero(self.stk_object.target_bins[idx])/324 for idx in range(len(self.stk_object.targets))])
        times = np.array([self.stk_object.target_times[idx]/86400 for idx in range(len(self.stk_object.targets))])
        
        percentage = np.average(percentages)
        time = np.average(times)
        std_time = np.std(times)

        if percentage < 25:
            if gen >= 1:
                percentage = (percentage/(2*gen - 1)).round(2)
            else:
                percentage.round(2)
        else:
            percentage = 25.00
        
        return percentage,n_sats,n_planes

    
    def Load_Individual(self,Individual=[0,0,0,0,0,0]):
        Alt = Individual[0]
        Inc = Individual[1]
        initial_raan = Individual[2]
        delta_raan = Individual[3]
        n_planes = int(Individual[5])
        n_sats = int(Individual[4])
        
        if n_planes > n_sats:
            n_planes = n_sats

        Asc = initial_raan
        file = open(f"../../Input_Files/Satellites_File.txt","w")
        file.write("Per,Apo,Inc,Asc,Loc\n")
        sats = n_sats*[1]
        planes = np.array_split(sats,n_planes)
        i=1
        for plane in planes:
            Loc = 0
            for sat in plane:
                file.write(f"{Alt},{Alt},{Inc},{round(Asc%360,4)},{round(Loc,4)}\n")
                if len(plane)>1: Loc += 360/len(plane)
            if len(planes)>1:Asc -= i*((-1)**(i))*delta_raan
            i+=1
        file.close()
opt = Optimizer2(stk_object,n_pop=10,n_gen=10)

print("Beginning Optimization")
hof = opt.run(read=False,enable_print=True)


-- Generation 3 --
      avg       std   min   max
0  20.371  8.400458  2.64  25.0
1  11.200  1.077033  9.00  12.0
2   8.500  2.202272  3.00  11.0
- Computing_AzEl [==========] 180/180 [100%] in 2.2s (81.20/s)                  
- Computing_AzEl [===>!     ] (!) 53/150 [35%] in 3.9s (14.16/s)                ▅▃▁ 24/150 [16%] in 1s (~4s, 34.7/s) [===>      ] ▇▇▅ 50/150 [33%] in 3s (~7s, 15.2/s) [===>      ] ▄▂▂ 52/150 [35%] in 4s (~7s, 14.6/s) 


In [ ]:
opt.Load_Individual(hof[0])

In [ ]:
pd.DataFrame([hof[0]], columns = 'Alt,Inc,Init_RAAN,Delta_RAAN,N_sats,N_planes'.split(','))